# Types and Dispatch in Julia

One of the most important goals of high-level languages is to provide *polymorphism*: the ability for the same code to operate on different kinds of values.

Julia uses a vocabulary of *types* for this purpose. Types play the following roles:

- Describe "what kind of thing is this"
- Describe the representation of a value
- Driving *dispatch*: selecting one of several pieces of code
- Driving *specialization*: code is optimized by assuming values have certain types

## Describing values

In [ ]:
typeof(3)

In [ ]:
sizeof(Int64)

In [ ]:
Int64.size

In [ ]:
isbits(Int64)

In [ ]:
Int64.mutable

In [ ]:
supertype(Int64)

In [ ]:
supertype(Signed)

In [ ]:
supertype(Integer)

In [ ]:
supertype(Real)

In [ ]:
supertype(Number)

## Dispatch

In [ ]:
f(a, b::Any)              = "fallback"
f(a::Number, b::Number)   = "a and b are both numbers"
f(a::Number, b)           = "a is a number"
f(a, b::Number)           = "b is a number"
f(a::Integer, b::Integer) = "a and b are both integers"

In [ ]:
methods(f)

In [ ]:
f(1.5, 2)

In [ ]:
f(1, "string")

In [ ]:
f(1, 2)

In [ ]:
f(1, 2, 3)

In [ ]:
methods(+)

## "Diagonal" dispatch

In [ ]:
d{T}(x::T, y::T) = "same type"
d(x, y) = "different types"

In [ ]:
d(1, 1)

In [ ]:
d(1, 2.0)

## Variadic (or varargs) methods

In [ ]:
v(x...) = (x, "zero or more")

In [ ]:
v(x, xs...) = (xs, "one or more")

In [ ]:
v()

In [ ]:
v(1)

In [ ]:
v(1, 2, 3, 4)

## Parametric types

Types can have nested structure.

In [ ]:
typeof(rand(2,2))

In [ ]:
typeof((1,2.0))

In [ ]:
typeof([(1,2.0)])

In [ ]:
Array{Float64,2} <: Array

In [ ]:
Array{Float64,2} <: Array{Any}

In [ ]:
Array{Float64,2} <: Array{Float64}

## Variadic tuple types

Conceptually, the arguments in every function call have a tuple type.

Julia is a tuple-slinging machine!

In [ ]:
foo(a::Array, Is::Int...) = 0

In [ ]:
vt = Tuple{Array, Vararg{Int}}

In [ ]:
isa(([1],1,2,3), vt)

In [ ]:
isa(([1],1,.02,3), vt)

## Union types

In [ ]:
isa(1, Union{Int,String})

In [ ]:
isa("hello", Union{Int,String})

## Functions of types

In [ ]:
elty{T}(::Type{Vector{T}}) = T

In [ ]:
elty(Vector{Int16})

Useful for calling `convert`, etc.

## Specialization in action

Internally, the compiler generates specializations for particular types.

Method signatures are Tuple types.

Example: For a 3-argument function `f`, the compiler might decide to generate a specialization for `Tuple{Int, Any, Int}`, if for some reason the second argument isn't important.

In [ ]:
addall(t) = +(t...)  # "splat"

In [ ]:
@code_typed addall((1,2))

In [ ]:
@code_typed addall((1,2,3))

In [ ]:
function alltrue(f, itr)
    @inbounds for x in itr
        f(x) || return false
    end
    return true
end

In [ ]:
@which isinteger(1)

In [ ]:
@code_typed alltrue(isinteger, [1,2,3])

In [ ]:
@code_llvm alltrue(isinteger, [1,2,3])

## Dispatch, specialization, and performance

Dynamic dispatch is traditionally considered "slow".

Instead of a `call` instruction, you need to do a table lookup procedure first.

However:
1. If types are known, the call target can be looked up at compile time.
2. The cost of dynamic dispatch is well worth it *if* you're dispatching to an optimized kernel.

## What to specialize on?

We can't specialize on *everything* because it would take too long and generate too much code.

There's no fully general and automatic approach.

We specialize on types. That's a reasonable default. If the default's not good enough, move more information into types!

A classic: specializing on the value of an integer.

In [ ]:
iota{N}(::Val{N}) = ntuple(identity, Val{N})

In [ ]:
iota(Val{4}())

In [ ]:
@code_llvm ntuple(identity, Val{4})

In [ ]:
iota(n::Integer) = iota(Val{n}())

In [ ]:
iota(3)

In [ ]:
iota(rand(1:5))  # dynamic dispatch to specialized code!

## Application to our favorite problem (circular shifting of course!)

Circular shift where `s` == 1 is easy!

In [ ]:
function circularshift(X::AbstractVector, s::Integer)
    n = length(X)
    s = mod(s,n)
    return vcat(view(X, n-s+1:n), view(X, 1:n-s))
end

In [ ]:
function circularshift1!(X::AbstractVector)
    n = length(X)
    temp = X[n]
    copy!(X, 2, X, 1, n-1)
    X[1] = temp
    return X
end

In [ ]:
x = rand(1000000);

In [ ]:
using BenchmarkTools

In [ ]:
@benchmark circularshift(x, 1)

In [ ]:
@benchmark circularshift1!(x)

#### Now let's generalize this from s == 1 to *small s*

In [ ]:
function circularshiftN!{S}(X::AbstractVector, ::Val{S})
    n = length(X)
    temp = ntuple(i->X[n-S+i], Val{S})
    copy!(X, S+1, X, 1, n-S)
    @inbounds for i = 1:S
        X[i] = temp[i]
    end
    return X
end

In [ ]:
@benchmark circularshift(x, 4)

In [ ]:
@benchmark circularshiftN!(x, Val{4}())

# "Stupid Dispatch Tricks"

## Trick 1: processing arguments recursively

The compiler's optimizations can be exploited to move parts of your own computations to compile time (thus saving time at run time). The general idea is to represent more information within types, instead of using values.

Example: drop the first element of a tuple.

In [ ]:
tuple_tail1(t) = t[2:end]

In [ ]:
tuple_tail1((1,2,"hi"))

In [ ]:
@code_typed tuple_tail1((1,2,"hi"))

Not good. Key information is represented as integers, and when the compiler sees an integer it generally assumes it doesn't know its value.

- The compiler counts 1, infinity
- The compiler can match things but cannot do arithmetic or comparisons
- It's very good at knowing the types of function arguments

In [ ]:
argtail(a, rest...) = rest
tupletail(t) = argtail(t...)

In [ ]:
tupletail((1,2,"hi"))

In [ ]:
@code_typed tupletail((1,2,"hi"))

### Exercise

Write a type-inferable function to...

1. reverse a tuple
1. take every other element of a tuple
2. interleave the elements of two tuples

### Real-ish example: computing the shape of an indexing operation

In [ ]:
index_shape(a::Array, idxs) = ish(a, 1, idxs...)

ish(a, i,   ::Real...)         = ()
ish(a, i,   ::Colon,  rest...) = (size(a,i), ish(a,i+1,rest...)...)
ish(a, i, iv::Vector, rest...) = (length(iv), ish(a,i+1,rest...)...)
ish(a, i,   ::Real,   rest...) = ish(a,i+1,rest...)

In [ ]:
index_shape(rand(3,4,5), (1,:,[1,2]))

In [ ]:
index_shape(rand(3,4,5), (:,2,[1,2,1,2,1,2,1,2]))

## Trick 2: look up "trait" values and re-dispatch
A.K.A. The THTT (Tim Holy Traits Trick)

In [ ]:
# Sample trait
abstract IteratorSize
immutable SizeUnknown <: IteratorSize end
immutable HasLength <: IteratorSize end
immutable HasShape <: IteratorSize end
immutable IsInfinite <: IteratorSize end

Now we can define a method that says which value of the trait a certain type has.

This is like using dispatch as a lookup table to find out properties of a combination of values.

In [ ]:
iteratorsize{T<:AbstractArray}(::Type{T}) = HasShape()

iteratorsize{I1,I2}(::Type{Zip2{I1,I2}}) = zip_iteratorsize(iteratorsize(I1),iteratorsize(I2))

zip_iteratorsize(a, b) = SizeUnknown()
zip_iteratorsize{T}(isz::T, ::T) = isz
zip_iteratorsize(::HasLength, ::HasShape) = HasLength()
zip_iteratorsize(::HasShape, ::HasLength) = HasLength()

In [ ]:
# `collect` gives you all the elements from an iterator as an array
collec(itr) = _collec(itr, eltype(itr), Base.iteratorsize(itr))

In [ ]:
function _collec(itr, T, ::Base.HasLength)
    a = Array{T,1}(length(itr))
    i = 0
    for x in itr
        a[i+=1] = x
    end
    return a
end

In [ ]:
function _collec(itr, T, ::Base.SizeUnknown)
    a = Array{T,1}(0)
    for x in itr
        push!(a, x)
    end
    return a
end

In [ ]:
# We use something similar for type promotion
promote_type(Int64, Float64)